In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../src')

import math
import cadquery as cq
from ocp_vscode import show

from chapel2.dome_generator import (
    generate_dome_with_hubs,
    generate_chapel_dome,
)

from chapel2.geometry import (
    generate_honeycomb_dome,
    normalize, sub, add, scale, cross, norm, build_vertex_to_edges_map
)

from chapel2.hubs import (
    compute_hub_geometry,
    create_hub_by_style,
)

# Chapel of MOOP - 8ft 3V Honeycomb Dome

Configuration:
- **Style**: Honeycomb (Hex/Pent)
- **Radius**: 8 ft
- **Frequency**: 3V
- **Hubs**: Tapered Prism
- **Struts**: Cuboid
- **Windows**: Hexagonal/Pentagonal Plates

In [6]:
# Generate Chapel dome with tapered prism hubs
print("Generating Chapel dome with TAPERED PRISM hubs...")

struts_prism, hubs_prism, windows_prism, info_prism = generate_dome_with_hubs(
    radius_cm=8.0 * 30.48,
    frequency=3,
    strut_width=3.5 * 2.54,
    strut_depth=3.5 * 2.54,
    dome_style="honeycomb",
    hub_style="tapered_prism",
    strut_style="cuboid",
    generate_windows=True,
    window_plate_depth=1 * 2.54,
)

print(f"  Struts: {info_prism['num_struts_generated']}")
print(f"  Hubs: {info_prism['num_hubs_generated']}")
print(f"  Windows: {info_prism['num_windows_generated']}")

# Visualize tapered prism dome
print("Chapel Dome with Tapered Prism Hubs and Windows:")
show(struts_prism, hubs_prism, windows_prism)

Generating Chapel dome with TAPERED PRISM hubs...
  Struts: 264
  Hubs: 168
  Windows: 89
Chapel Dome with Tapered Prism Hubs and Windows:
cc+


## Manufacturability Analysis

Analyzing unique part counts for manufacturing.

In [ ]:
from collections import Counter

def analyze_unique_struts(vertices, edges, dome_center, 
                          length_tol_cm=0.1, angle_tol_deg=0.5):
    """
    Analyze struts and group by similar dimensions.
    
    For cuboid struts: group by length only (ends are perpendicular cuts)
    For miter-cut struts: group by length + cut angles at both ends
    """
    strut_signatures = []
    
    for v1_idx, v2_idx in edges:
        v1 = vertices[v1_idx]
        v2 = vertices[v2_idx]
        
        # Strut length
        length = norm(sub(v2, v1))
        length_rounded = round(length / length_tol_cm) * length_tol_cm
        
        # Cuboid struts - just length matters
        signature = (length_rounded,)
        
        strut_signatures.append(signature)
    
    # Count unique signatures
    signature_counts = Counter(strut_signatures)
    
    return {
        'total_struts': len(edges),
        'unique_types': len(signature_counts),
        'signature_counts': dict(signature_counts),
    }


def analyze_unique_hubs(vertices, edges, vertex_to_edges, dome_center, 
                        angle_tol_deg=1.0):
    """
    Analyze hubs and group by similar configurations.
    
    Hubs are characterized by:
    - Number of struts
    - Angles between adjacent struts (sorted)
    """
    hub_signatures = []
    
    for v_idx, edge_indices in vertex_to_edges.items():
        vertex = vertices[v_idx]
        num_struts = len(edge_indices)
        
        if num_struts < 2:
            continue
        
        # Get strut directions
        directions = []
        for edge_idx in edge_indices:
            v1_idx, v2_idx = edges[edge_idx]
            other_idx = v2_idx if v1_idx == v_idx else v1_idx
            other = vertices[other_idx]
            direction = normalize(sub(other, vertex))
            directions.append(direction)
        
        # Compute angles between all pairs of struts
        angles = []
        for i in range(len(directions)):
            for j in range(i+1, len(directions)):
                d1, d2 = directions[i], directions[j]
                cos_angle = d1[0]*d2[0] + d1[1]*d2[1] + d1[2]*d2[2]
                angle_deg = math.degrees(math.acos(max(-1, min(1, cos_angle))))
                angle_rounded = round(angle_deg / angle_tol_deg) * angle_tol_deg
                angles.append(angle_rounded)
        
        # Sort angles to create a canonical signature
        angles_sorted = tuple(sorted(angles))
        
        signature = (num_struts, angles_sorted)
        hub_signatures.append(signature)
    
    signature_counts = Counter(hub_signatures)
    
    return {
        'total_hubs': len(hub_signatures),
        'unique_types': len(signature_counts),
        'signature_counts': dict(signature_counts),
    }


In [ ]:
# Run Analysis using the geometry returned from generation

print("="*70)
print("MANUFACTURABILITY ANALYSIS - CHAPEL DOME (8ft, 3V)")
print("="*70)

vertices = info_prism['vertices']
edges = info_prism['edges']
dome_center = (0, 0, 0)

# Re-build vertex map for analysis
vertex_to_edges = build_vertex_to_edges_map(edges)

# Filter for real hubs (those above the cutoff)
# We know portion is 0.5, so cutoff is Y=0
cutoff_y = -0.1 # tolerance
real_vertex_indices = {i for i, v in enumerate(vertices) if v[1] >= cutoff_y}

# Filter vertex_to_edges to only include real vertices
# But keep their edges (which might connect to ghost vertices)
vertex_to_edges_filtered = {
    v: es for v, es in vertex_to_edges.items() if v in real_vertex_indices
}

# Hub Analysis
hub_analysis = analyze_unique_hubs(
    vertices, edges, vertex_to_edges_filtered, dome_center
)

print(f"\nHub Analysis:")
print(f"  Total hubs: {hub_analysis['total_hubs']}")
print(f"  Unique hub types: {hub_analysis['unique_types']}")
print(f"\n  Hub types (struts, angles) -> count:")
for sig, count in sorted(hub_analysis['signature_counts'].items()):
    num_struts = sig[0]
    angles = sig[1]
    angles_str = ", ".join(f"{a:.0f} deg" for a in angles)
    print(f"    {num_struts}-strut hub [{angles_str}]: {count} pieces")

# Strut Analysis
strut_analysis = analyze_unique_struts(vertices, edges, dome_center)
print(f"\nStrut Analysis:")
print(f"  Total struts: {strut_analysis['total_struts']}")
print(f"  Unique strut types: {strut_analysis['unique_types']}")
print(f"\n  Cut list (length -> count):")
for sig, count in sorted(strut_analysis['signature_counts'].items()):
    length_cm = sig[0]
    length_in = length_cm / 2.54
    print(f"    {length_cm:.1f} cm ({length_in:.1f} in): {count} pieces")

In [ ]:
# Detailed Cut List
print("="*70)
print("DETAILED CUT LIST - CUBOID STRUTS")
print("="*70)
print(f"\nStrut cross-section: 3.5 in x 3.5 in")
print(f"Cut type: Perpendicular (90 deg) at both ends")
print(f"\n{'Type':<6} {'Length (cm)':<12} {'Length (in)':<12} {'Count':<8} {'Notes'}")
print("-"*70)

for i, (sig, count) in enumerate(sorted(strut_analysis['signature_counts'].items()), 1):
    length_cm = sig[0]
    length_in = length_cm / 2.54
    print(f"A{i:<5} {length_cm:<12.1f} {length_in:<12.1f} {count:<8} Square cut both ends")

total_length_cm = sum(sig[0] * count for sig, count in strut_analysis['signature_counts'].items())
total_length_ft = total_length_cm / 30.48
print("-"*70)
print(f"TOTAL: {strut_analysis['total_struts']} struts, {total_length_ft:.0f} linear feet of material")